In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg , count

In [3]:
#intilize SparkSesion
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

#loading the data
flights = spark.read.format("csv").option("header", "true").load("/content/drive/MyDrive/Colab Notebooks/etds/departuredelays.csv")

In [4]:
#creatring a temp view of the dataframe
flights.createOrReplaceTempView("flights")


In [5]:
#perform analysis using spark sql
#example 1: average delay by origin airport

avg_delay_by_origin = spark.sql("""SELECT origin, AVG(delay) as avg_delay
                                FROM flights
                                GROUP BY origin
                                ORDER BY avg_delay DESC
                                LIMIT 10
                                """)

In [6]:
avg_delay_by_origin.show()

+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+



In [8]:
#example 2: total flights and average dealy by day of week
flights_by_day = spark.sql("""SELECT date, COUNT(*) as total_flights, AVG(delay) as avg_delay
                                FROM flights
                                GROUP BY date
                                ORDER BY date
                                LIMIT 7
                                """)

In [10]:
flights_by_day.show()

+--------+-------------+------------------+
|    date|total_flights|         avg_delay|
+--------+-------------+------------------+
|01010005|            1|              -8.0|
|01010010|            1|              -6.0|
|01010020|            2|              -1.0|
|01010023|            1|              14.0|
|01010025|            2|              15.0|
|01010029|            1|              49.0|
|01010030|            3|-5.666666666666667|
+--------+-------------+------------------+



In [11]:
#example 3: top 5 routes with the highest total delay
top_delayed_routes = spark.sql("""SELECT origin, destination,
                                 SUM(delay) as total_delay,
                                 COUNT(*) as flight_count
                                FROM flights
                                GROUP BY origin, destination
                                ORDER BY total_delay DESC
                                LIMIT 5
                                """)

In [12]:
top_delayed_routes.show()

+------+-----------+-----------+------------+
|origin|destination|total_delay|flight_count|
+------+-----------+-----------+------------+
|   LAX|        SFO|    51844.0|        3198|
|   ORD|        SFO|    41653.0|        1731|
|   SFO|        LAX|    40798.0|        3232|
|   LGA|        ATL|    35761.0|        2500|
|   JFK|        LAX|    35755.0|        2720|
+------+-----------+-----------+------------+



In [13]:
#showing results

print("🛫 ✈️  Top 10 Origins by Average Delay:")
avg_delay_by_origin.show()

print("\n📅 ⏱️  Flights and Average Delay by Day (First Week):")
flights_by_day.show()

print("\n🛣️ 🔥 Top 5 Routes with Highest Total Delay:")
top_delayed_routes.show()

🛫 ✈️  Top 10 Origins by Average Delay:
+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+


📅 ⏱️  Flights and Average Delay by Day (First Week):
+--------+-------------+------------------+
|    date|total_flights|         avg_delay|
+--------+-------------+------------------+
|01010005|            1|              -8.0|
|01010010|            1|              -6.0|
|01010020|            2|              -1.0|
|01010023|            1|              14.0|
|01010025|            2|              15.0|
|01010029|            1|              49.0|
|01010030|            3|-5.666666666666667|
+--------+-------------+------------------+


🛣️ 🔥 Top 5 Routes with Highe

In [14]:
spark.stop()

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, hour, month

In [16]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Extended Flights Analysis") \
    .getOrCreate()

In [17]:
#loading
flights = spark.read.format("csv").option("header", "true").load("/content/drive/MyDrive/Colab Notebooks/etds/departuredelays.csv")

In [18]:
# Create a temporary view of the DataFrame
flights.createOrReplaceTempView("flights")


In [19]:
#exAMPOLE 4: AVERAGE DELAY BY ORIGIN AIRPORT(T -10)
avg_delay_by_origin = spark.sql("""
    SELECT origin, AVG(delay) as avg_delay
    FROM flights
    GROUP BY origin
    ORDER BY avg_delay DESC
    LIMIT 10
  """)

In [21]:
#example 5: T- 5 busiest routues
busiest_routes = spark.sql("""
    SELECT origin, destination, COUNT(*) as flight_count
    FROM flights
    GROUP BY origin, destination
    ORDER BY flight_count DESC
    LIMIT 5
""")

In [23]:
#example 6: monthly flights trends
monthly_trends= spark.sql("""
                          SELECT
                              SUBSTRING(CAST(date AS STRING) ,4,4) as month,
                              COUNT(*) as total_flights,
                              AVG(delay) as avg_delay
                          FROM flights
                          GROUP BY SUBSTRING(CAST(date AS STRING) ,4,4)
                          ORDER BY month
                          """)

In [24]:
#example 7: percentage of dealyed flights by origin
delayed_percentage = spark.sql("""
                                SELECT
                                    origin,
                                    COUNT(*) as total_flights,
                                    SUM(CASE WHEN delay > 0 THEN 1 ELSE 0 END) as delayed_flights,
                                    (SUM(CASE WHEN delay > 0 THEN 1 ELSE 0 END) *100.0) / COUNT(*) as delay_percentage
                                FROM flights
                                GROUP BY origin
                                ORDER BY delay_percentage DESC
                                LIMIT 10
                                    """)

In [26]:
#example 8: average delay by hour of day
delay_by_hour = spark.sql("""
                        SELECT
                            CAST(SUBSTRING(CAST(date AS STRING), 10, 2) AS INT) AS hour,
                            AVG(delay) AS avg_delay
                        FROM flights
                        GROUP BY SUBSTRING(CAST(date AS STRING),10,2)
                        ORDER BY hour
                        """)

In [27]:
# Display results with emojis for better readability

print("🛫 ✈️ Top 10 Origins by Average Delay:")
avg_delay_by_origin.show()

print("\n🚦📈 Top 5 Busiest Routes:")
busiest_routes.show()

print("\n📅📊 Monthly Flight Trends:")
monthly_trends.show()

print("\n⏳📉 Top 10 Origins by Percentage of Delayed Flights:")
delayed_percentage.show()

print("\n🕒⏱️ Average Delay by Hour of Day:")
delay_by_hour.show()


🛫 ✈️ Top 10 Origins by Average Delay:
+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+


🚦📈 Top 5 Busiest Routes:
+------+-----------+------------+
|origin|destination|flight_count|
+------+-----------+------------+
|   SFO|        LAX|        3232|
|   LAX|        SFO|        3198|
|   LAS|        LAX|        3016|
|   LAX|        LAS|        2964|
|   JFK|        LAX|        2720|
+------+-----------+------------+


📅📊 Monthly Flight Trends:
+-----+-------------+--------------------+
|month|total_flights|           avg_delay|
+-----+-------------+--------------------+
| 0000|           11|  -5.090909090909091|
| 0001|            6| -2.166666666666

In [28]:
spark.stop()